# Group Members:

Saurabh Sisodia (PGID: 11915035)

Sivapradeep Katiki (PGID: 11915024)

Vikaskashyap Konda (PGID: 11915047)

# Zomato Webscraping Assignment


Import all necessary packages:

In [ ]:
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import os
import csv
from math import ceil
import re

- Initialize the Selenium Webdriver for scraping the data from zomato website.
- Navigate to "https://www.zomato.com/Hyderabad"
- Search by keyword 'Gachibowli, Hyderabad'

In [ ]:
#driver = webdriver.Chrome('C:/Users/xyz/Documents/ISBCBA/Term1/Data Collection/LEC 2-2019-04-12/LEC 2/data/chromedriver.exe')
driver = webdriver.Firefox(executable_path=r'C:/Users/xyz/Documents/ISBCBA/Term1/Data Collection/geckodriver-v0.24.0-win64/geckodriver.exe')
driver.get("https://www.zomato.com/Hyderabad")
assert "Zomato" in driver.title
elem = driver.find_element_by_css_selector('#keywords_input')
elem.clear()
elem.send_keys("Gachibowli, Hyderabad")
time.sleep(5)
elem.send_keys(Keys.RETURN)  
time.sleep(10)

Define the restaurant dataframe to store restaurant data from zomato website

In [ ]:
df_restaurant = pd.DataFrame(columns=['Name','Link','Timing','Rating','Vote','Cuisine','Cost','Collection'])

Scrape the zomato website and collect the restaurant information

We will be scraping first 100 restaurants in Gachibowli area.

- Capture basic information of a restaurant
    - Restaurant Name
    - Restaurant Link
    - Restaurant Timing
    - Restaurant Vote
    - Restaurant Cuisine
    - Restaurant Cost
    - Restaurant Collections
- Save the information of each restaurant in a Dataframe 'df_restaurant'
- Navigate through first 10 pages by clicking on next page button (as we want to capture details for first 100 restaurants)


In [ ]:
###############################################################################
###Get Restaurant details######################################################
###############################################################################
for page in range(1,11):
    time.sleep(5)
    print('------loading page'+str(page)+'-----------')
    html = driver.page_source
    soup = BeautifulSoup(html,features="lxml")
    ###Capturing restaurant details
    for i in soup.find_all('article',class_='search-result'):
        restaurant_dict={}
        try:
            Name = str(i.find('a',class_='result-title').get('title')).replace(', Gachibowli','')
        except:
            Name=''
        #Getting the link of the resturant to capture user reviews from the restaurant page later.
        try:
            Link = i.find('a',class_='result-title').get('href')
        except:
            Link=''
        try:
            Timing = i.find('div',class_='res-timing clearfix').get('title')
        except:
            Timing=''
        try:
            Rating = i.find('div',class_='res-rating-nf').string.strip()
        except:
            Rating=''
        try:
            Vote = i.find('span',class_=re.compile("rating-votes-div-")).string.strip()
        except:
            Vote = '0'
        #There are multiple cuisine for a restaurant, hence iterating over list to get all cuisines
        try:
            Cuisine=''
            for j in i.find_all('span',class_='col-s-11 col-m-12 nowrap pl0'):
                for k in j.find_all('a'):
                    Cuisine = k.string + ', ' + Cuisine
            Cuisine = Cuisine.strip()
            Cuisine = Cuisine[:len(Cuisine)-1]
        except:
            Cuisine=''
        try:
            Cost = i.find('span',class_='col-s-11 col-m-12 pl0').string.strip()
        except:
            Cost=''
        #Restaurant can be tagged under multiple collections, hence iterating over the list to get all collection tags
        try:
            Collection=''
            for j in i.find_all('div',class_='res-collections clearfix'):
                for k in j.find_all('a'):
                    Collection = Collection + ', ' + k.string
            Collection = Collection[2:]
        except:
            Collection=''
            
        restaurant_dict['Name'] = Name
        restaurant_dict['Link'] = Link
        restaurant_dict['Rating'] = Rating
        restaurant_dict['Vote'] = Vote
        restaurant_dict['Cuisine'] = Cuisine
        restaurant_dict['Cost'] = Cost
        restaurant_dict['Collection'] = Collection
        restaurant_dict['Timing'] = Timing
        df_restaurant = df_restaurant.append(restaurant_dict,ignore_index=True)
    if(page==1):
        driver.find_element_by_xpath('//*[@id="search-results-container"]/div[2]/div[1]/div[2]/div/div/a[7]').click()
    elif(page<=5):
        driver.find_element_by_xpath('//*[@id="search-results-container"]/div[2]/div[1]/div[2]/div/div/a[8]').click()
    else:
        driver.find_element_by_xpath('//*[@id="search-results-container"]/div[2]/div[1]/div[2]/div/div/a[9]').click()
    time.sleep(10)


- Get the current working directory
- Change the working directory to desired location
- Create a new csv to write the information with following columns
    - Restaurant Name
    - Cuisine
    - Cost
    - Timing
    - Rating
    - Vote
    - Collection
    - Reviewer Name
    - Review Text
    - Review Time
    - Total number of reviews given by reviewer
    - Number of followers of reviewer
    - Number of photos added by reviewer


In [ ]:
# Creating a csv file at the given location in order to capture and save the restaurant details in a CSV file.
os.getcwd()
os.chdir("C:\\Users\\xyz\\Documents\\ISBCBA\\Term1\\Data Collection\\MyPractice")
os.listdir()
csv_file = open("zomato_25May_final.csv",'w',encoding ='UTF-8',newline = "")  
writer = csv.writer(csv_file)
writer.writerow(
    ['Restaurant_Name','Cuisine','Cost','Restaurant_Timing','Restaurant_Rating','Vote','Collection',
     'Reviewer_Name','Review_text','Review_time','NoOfReview','NoOfFollower','NoOfPhoto'])

Define function to get first 100 reviews provided by the users for tha restaurant

- The function navigates to the restaurant page via selenium webdriver
- Checks the total number of reviews for the restaurant
- Click on to get 'All reviews' for the restaurant
- The function scrapes for first 100 reviews (from latest to older ones)
- The function picks up follwing information:
    - Reviewer Name
    - Reviewer Text : Some times the review text is large, so the function checks for 'Read More' button, to load the entire review text before scraping the data
    - Reviewer Follower : Number of followers of the reviewer
    - Reviewer Reviews : Total number of reviews provided by the user(reviewer)
    - Reviewer Photos : Number of photos added by the reviewer
    - Reviewer Time : Timestamp when the review was added by the reviewer
- Function saves the information in the dictionary
- Appends the dictionary to the the 'CSV' file

In [ ]:

def GetReviews(Link):
    driver.get(Link)
    time.sleep(10)
    Review_counts = int(driver.find_element_by_xpath('//*[@id="selectors"]/a[2]/span').text)
    print(Review_counts)
    #wait for 10 sec until element is clickable
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'.//a[@data-sort="reviews-dd"]'))).click()
    time.sleep(5)
    #If review count is more than 101 limit the scraping reviews to 100
    if(int(Review_counts) > 101):
        Review_counts = 101
    for i in range(1,Review_counts):
        try:
            # Creating an empty dictionary
            dict1={}
            # Saving restaurant details from the restaurant dataframe to dictionary 
            dict1['Restaurant_Name'] = df_restaurant['Name'][j]
            dict1['Cuisine'] = df_restaurant['Cuisine'][j]
            dict1['Cost'] = df_restaurant['Cost'][j][1:] #ignoring Rupee currency#
            dict1['Restaurant_Timing'] = df_restaurant['Timing'][j]
            dict1['Restaurant_Rating'] = df_restaurant['Rating'][j]
            dict1['Vote'] = df_restaurant['Vote'][j]
            dict1['Collection'] = df_restaurant['Collection'][j]
            print('#############'+str(i)+'##################')
            
            # Check to click on 'Load more' button to load more reviews
            if(i%5==0):
                print('x')
                driver.find_element_by_xpath('//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[2]/div[1]/span[1]').click()
                time.sleep(10)
            #Capture the details from review container block 
            review = driver.find_element_by_xpath('//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[1]/div['+str(i)+']/div')
            try:
                print(review.find_element_by_xpath('.//div[@class = "header nowrap ui left"]').text)
                rev_name = review.find_element_by_xpath('.//div[@class = "header nowrap ui left"]').text
            except:
                rev_name = ''
            # Xpath for the 1st review comment is different from all other review comments,
            # Hence a check if it is 1st review comment or not
            # For some reviews the review text is large, hence need to check for 'read more' option in order to 
            # load full review text
            try:
                if(i==1):
                    if(driver.find_element_by_xpath('//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[1]/div[1]/div/div[1]/div[4]').text == ''):
                        print('***********1********')
                        if(driver.find_element_by_xpath('//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[1]/div[1]/div/div[1]/div[5]/div/a').text == 'read more'):
                            url='//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[1]/div[1]/div/div[1]/div[5]/div/a'
                            driver.find_element_by_xpath(url).click()
                            time.sleep(5)
                    print(driver.find_element_by_xpath('//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[1]/div[1]/div/div[1]/div[4]').text)
                    rev_text = driver.find_element_by_xpath('//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[1]/div[1]/div/div[1]/div[4]').text
                else:
                    if (driver.find_element_by_xpath('//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[1]/div['+str(i)+']/div/div[1]/div[3]').text == ''):
                        print('***********')
                        if(driver.find_element_by_xpath('//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[1]/div['+str(i)+']/div/div[1]/div[4]/div/a').text == 'read more'):
                            print('****************')
                            url = '//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[1]/div['+str(i)+']/div/div[1]/div[4]/div/a'
                            driver.find_element_by_xpath(url).click()
                            time.sleep(5)
                    print(driver.find_element_by_xpath('//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[1]/div['+str(i)+']/div/div[1]/div[3]').text)
                    rev_text = driver.find_element_by_xpath('//*[@id="reviews-container"]/div[1]/div[3]/div/div/div[1]/div['+str(i)+']/div/div[1]/div[3]').text
            except:
                rev_text=''
            # check the number of followers and number of reviews reviewers
            # There are some cases where user might have no followers, in that case there will be only number of reviews option
            # Hence to consider followers and number of reviewers separately
            try:
                rev_followers = review.find_element_by_xpath('.//span[@class = "grey-text fontsize5 nowrap"]').text
                rev_followers = rev_followers.split()
                rev_reviews  = int(rev_followers[0])
            except:
                rev_reviews = 0
            print(rev_reviews)
            try:
                rev_followers = int(rev_followers[3])
            except:
                rev_followers = 0
            print(rev_followers)
            try:
                review_photo = len(review.find_elements_by_xpath('.//div[@class="ui image js-heart-container"]'))
            except:
                review_photo = 0
            try:
                additional_photo = int(str(review.find_element_by_xpath('.//div[@class="overlay res-photo-thumbnail tac"]').text).split()[0][1:])
            except:
                additional_photo = 0
            rev_photos = review_photo+additional_photo
            print(rev_photos)
            try:
                print(review.find_element_by_xpath('.//time').get_attribute('datetime'))
                rev_datetime = review.find_element_by_xpath('.//time').get_attribute('datetime')
            except:
                rev_datetime=''
            #Saving the review information in dictionary    
            dict1['Reviewer_Name'] = rev_name
            dict1['Review_text'] = rev_text
            dict1['Review_time'] = rev_datetime
            dict1['NoOfReview'] = rev_reviews
            dict1['NoOfFollower'] = rev_followers
            dict1['NoOfPhoto'] = rev_photos
            #writing dictionary to the CSV file
            writer.writerow(dict1.values())
            print('**********************************')
        except:
            continue
    print('-------completed-----')


Run the review function for first 100 restaurants 

In [ ]:
for j in range(0,101):
    url=df_restaurant['Link'][j]
    print(url)
    GetReviews(url)

Close the CSV file and Webdriver

In [ ]:
csv_file.close()    
driver.close()